In [1]:
import numpy as np
import pandas as pd
import json
import os

from pandas.testing import assert_frame_equal

# define functions

In [2]:
def transferStrToJson(original_str):
    original_str = original_str.replace("'", '"')
    json_text = json.loads(original_str)
    return json_text

In [3]:
def transferOriginalToProcessed(original_df):
    original_df_temp = original_df.copy(deep = True)
    new_anomaly_processed_df = pd.DataFrame(columns = ["caseid", "Activity", "concept:name", "lifecycle:transition",
                                          "time:timestamp", "name", "order", "label"])
    original_unique_cases = original_df_temp['caseid'].unique()
    idx = 0
    for index, case in enumerate(original_unique_cases):
        print('--------------------------------------------------------')
        print('index=', index, 'case=', case, type(case))
        
        case_length = original_df_temp[original_df_temp['caseid'] == case].shape[0]
        order_list = original_df_temp[original_df_temp['caseid'] == case]['order'].to_list()
        timestamp_list = original_df_temp[original_df_temp['caseid'] == case]['timestamp'].to_list()
        name_list = original_df_temp[original_df_temp['caseid'] == case]['name'].to_list()
        label_list = original_df_temp[original_df_temp['caseid'] == case]['label'].to_list()
        
        anomaly_label = label_list[0]
        if anomaly_label == "normal":
            for i in range(case_length):
                new_anomaly_processed_df.at[idx, "caseid"]  =  index+1
                new_anomaly_processed_df.at[idx, "Activity"]  =  name_list[i]
                new_anomaly_processed_df.at[idx, "concept:name"]  =  name_list[i]
                new_anomaly_processed_df.at[idx, "lifecycle:transition"]  =  'complete'
                new_anomaly_processed_df.at[idx, "time:timestamp"]  =  timestamp_list[i]
                new_anomaly_processed_df.at[idx, "name"]  =  name_list[i]
                new_anomaly_processed_df.at[idx, "order"]  =  order_list[i]
                new_anomaly_processed_df.at[idx, "label"]  =  False
                idx = idx + 1
        else:
            # 处理异常信息
            json_anomaly_label = transferStrToJson(anomaly_label)
            anomaly_type = json_anomaly_label['anomaly']
            inserted_positions = []
            inserted_act_names = []
            inserted_positions_index = 0
            for i in range(case_length):
                new_anomaly_processed_df.at[idx, "caseid"]  =  index+1
                if anomaly_type == 'Insert':
                    # 可能在多个不同的位置插入随机活动，因此这是一个数组
                    inserted_positions = json_anomaly_label['attr']['indices']
                    inserted_act_names = [name_list[i] for i in range(len(name_list)) if i in inserted_positions ]
                    if i in inserted_positions:
                        new_anomaly_processed_df.at[idx, "Activity"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "concept:name"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "lifecycle:transition"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "time:timestamp"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "name"]  =  inserted_act_names[inserted_positions_index] ###################
                        new_anomaly_processed_df.at[idx, "concept:name"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "lifecycle:transition"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "time:timestamp"]  =  np.nan
                        inserted_positions_index = inserted_positions_index + 1
                    else:
                        new_anomaly_processed_df.at[idx, "Activity"]  =  name_list[i]
                        new_anomaly_processed_df.at[idx, "concept:name"]  =  name_list[i]
                        new_anomaly_processed_df.at[idx, "lifecycle:transition"]  =  'complete'
                        new_anomaly_processed_df.at[idx, "time:timestamp"]  =  timestamp_list[i]
                        new_anomaly_processed_df.at[idx, "name"]  =  name_list[i]
                elif anomaly_type == 'Replace':
                    anomaly_position = json_anomaly_label['attr']['indices'][0]
                    if i == anomaly_position:
                        new_anomaly_processed_df.at[idx, "Activity"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "concept:name"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "lifecycle:transition"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "time:timestamp"]  =  np.nan
                    else:
                        new_anomaly_processed_df.at[idx, "Activity"]  =  name_list[i]
                        new_anomaly_processed_df.at[idx, "concept:name"]  =  name_list[i]
                        new_anomaly_processed_df.at[idx, "lifecycle:transition"]  =  'complete'
                        new_anomaly_processed_df.at[idx, "time:timestamp"]  =  timestamp_list[i]
                    new_anomaly_processed_df.at[idx, "name"]  =  name_list[i]
                else:
                    new_anomaly_processed_df.at[idx, "Activity"]  =  name_list[i]
                    new_anomaly_processed_df.at[idx, "concept:name"]  =  name_list[i]
                    new_anomaly_processed_df.at[idx, "lifecycle:transition"]  =  'complete'
                    new_anomaly_processed_df.at[idx, "time:timestamp"]  =  timestamp_list[i]
                    new_anomaly_processed_df.at[idx, "name"]  =  name_list[i]
                new_anomaly_processed_df.at[idx, "order"]  =  order_list[i]
                new_anomaly_processed_df.at[idx, "label"]  =  True
                idx = idx + 1
                
    new_anomaly_processed_df['caseid'] = new_anomaly_processed_df['caseid'].astype(np.int64)
    new_anomaly_processed_df['order'] = new_anomaly_processed_df['order'].astype(np.int64)
    new_anomaly_processed_df['label'] = new_anomaly_processed_df['label'].astype(bool)
    
    return new_anomaly_processed_df

In [4]:
def transferOriginalToProcessedTest(original_df):
    original_df_temp = original_df.copy(deep = True)
    
    original_df_temp[['caseid']]= original_df_temp[['caseid']].values.astype(str)
    original_df_temp[['concept:name']]= original_df_temp[['concept:name']].values.astype(str)
    
    new_anomaly_processed_df = pd.DataFrame(columns = ["caseid", "Activity", "concept:name", "lifecycle:transition",
                                          "time:timestamp", "name", "order", "label"])
    original_unique_cases = original_df_temp['caseid'].unique()
    idx = 0
    for index, case in enumerate(original_unique_cases):
        print('--------------------------------------------------------')
#         if type(case) is float: # for dataset sepsis_case
#             case = np.nan
#             case_length = original_df_temp[original_df_temp['caseid'] == case].shape[0]
#             order_list = original_df_temp[original_df_temp['caseid'] == case]['order'].to_list()
#             timestamp_list = original_df_temp[original_df_temp['caseid'] == case]['timestamp'].to_list()
#             name_list = original_df_temp[original_df_temp['caseid'] == case]['name'].to_list()
#             label_list = original_df_temp[original_df_temp['caseid'] == case]['label'].to_list()
#         else:
        case_length = original_df_temp[original_df_temp['caseid'] == case].shape[0]
        order_list = original_df_temp[original_df_temp['caseid'] == case]['order'].to_list()
        timestamp_list = original_df_temp[original_df_temp['caseid'] == case]['timestamp'].to_list()
        name_list = original_df_temp[original_df_temp['caseid'] == case]['name'].to_list()
        label_list = original_df_temp[original_df_temp['caseid'] == case]['label'].to_list()
            
        print('index=', index, 'case=', case, type(case))
               
        print(type(label_list), len(label_list))
        
        anomaly_label = label_list[0]
        if anomaly_label == "normal":
            for i in range(case_length):
                new_anomaly_processed_df.at[idx, "caseid"]  =  index+1
                new_anomaly_processed_df.at[idx, "Activity"]  =  name_list[i]
                new_anomaly_processed_df.at[idx, "concept:name"]  =  name_list[i]
                new_anomaly_processed_df.at[idx, "lifecycle:transition"]  =  'complete'
                new_anomaly_processed_df.at[idx, "time:timestamp"]  =  timestamp_list[i]
                new_anomaly_processed_df.at[idx, "name"]  =  name_list[i]
                new_anomaly_processed_df.at[idx, "order"]  =  order_list[i]
                new_anomaly_processed_df.at[idx, "label"]  =  False
                idx = idx + 1
        else:
            # 处理异常信息
            json_anomaly_label = transferStrToJson(anomaly_label)
            anomaly_type = json_anomaly_label['anomaly']
            inserted_positions = []
            inserted_act_names = []
            inserted_positions_index = 0
            for i in range(case_length):
                new_anomaly_processed_df.at[idx, "caseid"]  =  index+1
                if anomaly_type == 'Insert':
                    # 可能在多个不同的位置插入随机活动，因此这是一个数组
                    inserted_positions = json_anomaly_label['attr']['indices']
                    inserted_act_names = [name_list[i] for i in range(len(name_list)) if i in inserted_positions ]
                    if i in inserted_positions:
                        new_anomaly_processed_df.at[idx, "Activity"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "concept:name"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "lifecycle:transition"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "time:timestamp"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "name"]  =  inserted_act_names[inserted_positions_index] ###################
                        new_anomaly_processed_df.at[idx, "concept:name"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "lifecycle:transition"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "time:timestamp"]  =  np.nan
                        inserted_positions_index = inserted_positions_index + 1
                    else:
                        new_anomaly_processed_df.at[idx, "Activity"]  =  name_list[i]
                        new_anomaly_processed_df.at[idx, "concept:name"]  =  name_list[i]
                        new_anomaly_processed_df.at[idx, "lifecycle:transition"]  =  'complete'
                        new_anomaly_processed_df.at[idx, "time:timestamp"]  =  timestamp_list[i]
                        new_anomaly_processed_df.at[idx, "name"]  =  name_list[i]
                elif anomaly_type == 'Replace':
                    anomaly_position = json_anomaly_label['attr']['indices'][0]
                    if i == anomaly_position:
                        new_anomaly_processed_df.at[idx, "Activity"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "concept:name"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "lifecycle:transition"]  =  np.nan
                        new_anomaly_processed_df.at[idx, "time:timestamp"]  =  np.nan
                    else:
                        new_anomaly_processed_df.at[idx, "Activity"]  =  name_list[i]
                        new_anomaly_processed_df.at[idx, "concept:name"]  =  name_list[i]
                        new_anomaly_processed_df.at[idx, "lifecycle:transition"]  =  'complete'
                        new_anomaly_processed_df.at[idx, "time:timestamp"]  =  timestamp_list[i]
                    new_anomaly_processed_df.at[idx, "name"]  =  name_list[i]
                else:
                    new_anomaly_processed_df.at[idx, "Activity"]  =  name_list[i]
                    new_anomaly_processed_df.at[idx, "concept:name"]  =  name_list[i]
                    new_anomaly_processed_df.at[idx, "lifecycle:transition"]  =  'complete'
                    new_anomaly_processed_df.at[idx, "time:timestamp"]  =  timestamp_list[i]
                    new_anomaly_processed_df.at[idx, "name"]  =  name_list[i]
                new_anomaly_processed_df.at[idx, "order"]  =  order_list[i]
                new_anomaly_processed_df.at[idx, "label"]  =  True
                idx = idx + 1
                
    new_anomaly_processed_df['caseid'] = new_anomaly_processed_df['caseid'].astype(np.int64)
    new_anomaly_processed_df['order'] = new_anomaly_processed_df['order'].astype(np.int64)
    new_anomaly_processed_df['label'] = new_anomaly_processed_df['label'].astype(bool)
    
    return new_anomaly_processed_df

# process original csv

In [7]:
original_dir = './original_csv/formal/'
processed_dir = './processed_csv/formal/'

original_csv_list = os.listdir(original_dir)  # get csv list
original_df_list = []
for csv_file in original_csv_list:
    csv_file_name = csv_file[:-4]
    print(csv_file, csv_file_name)
    original_df = pd.read_csv(original_dir + csv_file)
    processed_df = transferOriginalToProcessedTest(original_df) # for sepsis-case
#     processed_df = transferOriginalToProcessed(original_df)
    processed_df.to_csv(processed_dir + csv_file_name + '.csv', index = None)
    

SepsisCases-0.2-1.csv SepsisCases-0.2-1
--------------------------------------------------------
index= 0 case= A <class 'str'>
<class 'list'> 22
--------------------------------------------------------
index= 1 case= B <class 'str'>
<class 'list'> 12
--------------------------------------------------------
index= 2 case= C <class 'str'>
<class 'list'> 14
--------------------------------------------------------
index= 3 case= D <class 'str'>
<class 'list'> 13
--------------------------------------------------------
index= 4 case= E <class 'str'>
<class 'list'> 8
--------------------------------------------------------
index= 5 case= F <class 'str'>
<class 'list'> 10
--------------------------------------------------------
index= 6 case= G <class 'str'>
<class 'list'> 12
--------------------------------------------------------
index= 7 case= H <class 'str'>
<class 'list'> 13
--------------------------------------------------------
index= 8 case= I <class 'str'>
<class 'list'> 11
-------

--------------------------------------------------------
index= 87 case= JC <class 'str'>
<class 'list'> 15
--------------------------------------------------------
index= 88 case= KC <class 'str'>
<class 'list'> 13
--------------------------------------------------------
index= 89 case= LC <class 'str'>
<class 'list'> 21
--------------------------------------------------------
index= 90 case= MC <class 'str'>
<class 'list'> 14
--------------------------------------------------------
index= 91 case= NC <class 'str'>
<class 'list'> 7
--------------------------------------------------------
index= 92 case= OC <class 'str'>
<class 'list'> 22
--------------------------------------------------------
index= 93 case= PC <class 'str'>
<class 'list'> 5
--------------------------------------------------------
index= 94 case= QC <class 'str'>
<class 'list'> 15
--------------------------------------------------------
index= 95 case= RC <class 'str'>
<class 'list'> 13
------------------------------

--------------------------------------------------------
index= 164 case= IF <class 'str'>
<class 'list'> 17
--------------------------------------------------------
index= 165 case= JF <class 'str'>
<class 'list'> 8
--------------------------------------------------------
index= 166 case= KF <class 'str'>
<class 'list'> 15
--------------------------------------------------------
index= 167 case= LF <class 'str'>
<class 'list'> 5
--------------------------------------------------------
index= 168 case= MF <class 'str'>
<class 'list'> 2
--------------------------------------------------------
index= 169 case= NF <class 'str'>
<class 'list'> 28
--------------------------------------------------------
index= 170 case= OF <class 'str'>
<class 'list'> 3
--------------------------------------------------------
index= 171 case= PF <class 'str'>
<class 'list'> 15
--------------------------------------------------------
index= 172 case= QF <class 'str'>
<class 'list'> 16
-----------------------

--------------------------------------------------------
index= 242 case= II <class 'str'>
<class 'list'> 13
--------------------------------------------------------
index= 243 case= JI <class 'str'>
<class 'list'> 10
--------------------------------------------------------
index= 244 case= KI <class 'str'>
<class 'list'> 18
--------------------------------------------------------
index= 245 case= LI <class 'str'>
<class 'list'> 14
--------------------------------------------------------
index= 246 case= MI <class 'str'>
<class 'list'> 10
--------------------------------------------------------
index= 247 case= NI <class 'str'>
<class 'list'> 15
--------------------------------------------------------
index= 248 case= OI <class 'str'>
<class 'list'> 6
--------------------------------------------------------
index= 249 case= PI <class 'str'>
<class 'list'> 11
--------------------------------------------------------
index= 250 case= QI <class 'str'>
<class 'list'> 9
---------------------

--------------------------------------------------------
index= 318 case= HL <class 'str'>
<class 'list'> 15
--------------------------------------------------------
index= 319 case= IL <class 'str'>
<class 'list'> 13
--------------------------------------------------------
index= 320 case= JL <class 'str'>
<class 'list'> 10
--------------------------------------------------------
index= 321 case= KL <class 'str'>
<class 'list'> 5
--------------------------------------------------------
index= 322 case= LL <class 'str'>
<class 'list'> 22
--------------------------------------------------------
index= 323 case= ML <class 'str'>
<class 'list'> 29
--------------------------------------------------------
index= 324 case= NL <class 'str'>
<class 'list'> 6
--------------------------------------------------------
index= 325 case= OL <class 'str'>
<class 'list'> 14
--------------------------------------------------------
index= 326 case= PL <class 'str'>
<class 'list'> 6
----------------------

...

--------------------------------------------------------
index= 707 case= GAA <class 'str'>
<class 'list'> 16
--------------------------------------------------------
index= 708 case= HAA <class 'str'>
<class 'list'> 21
--------------------------------------------------------
index= 709 case= IAA <class 'str'>
<class 'list'> 9
--------------------------------------------------------
index= 710 case= JAA <class 'str'>
<class 'list'> 23
--------------------------------------------------------
index= 711 case= KAA <class 'str'>
<class 'list'> 5
--------------------------------------------------------
index= 712 case= LAA <class 'str'>
<class 'list'> 8
--------------------------------------------------------
index= 713 case= MAA <class 'str'>
<class 'list'> 14
--------------------------------------------------------
index= 714 case= NAA <class 'str'>
<class 'list'> 3
--------------------------------------------------------
index= 715 case= OAA <class 'str'>
<class 'list'> 37
--------------

--------------------------------------------------------
index= 791 case= MDA <class 'str'>
<class 'list'> 12
--------------------------------------------------------
index= 792 case= NDA <class 'str'>
<class 'list'> 15
--------------------------------------------------------
index= 793 case= ODA <class 'str'>
<class 'list'> 23
--------------------------------------------------------
index= 794 case= PDA <class 'str'>
<class 'list'> 16
--------------------------------------------------------
index= 795 case= QDA <class 'str'>
<class 'list'> 9
--------------------------------------------------------
index= 796 case= RDA <class 'str'>
<class 'list'> 10
--------------------------------------------------------
index= 797 case= SDA <class 'str'>
<class 'list'> 14
--------------------------------------------------------
index= 798 case= TDA <class 'str'>
<class 'list'> 10
--------------------------------------------------------
index= 799 case= UDA <class 'str'>
<class 'list'> 30
-----------

--------------------------------------------------------
index= 871 case= OGA <class 'str'>
<class 'list'> 13
--------------------------------------------------------
index= 872 case= PGA <class 'str'>
<class 'list'> 23
--------------------------------------------------------
index= 873 case= QGA <class 'str'>
<class 'list'> 7
--------------------------------------------------------
index= 874 case= RGA <class 'str'>
<class 'list'> 14
--------------------------------------------------------
index= 875 case= SGA <class 'str'>
<class 'list'> 16
--------------------------------------------------------
index= 876 case= TGA <class 'str'>
<class 'list'> 15
--------------------------------------------------------
index= 877 case= UGA <class 'str'>
<class 'list'> 15
--------------------------------------------------------
index= 878 case= VGA <class 'str'>
<class 'list'> 8
--------------------------------------------------------
index= 879 case= WGA <class 'str'>
<class 'list'> 36
------------

--------------------------------------------------------
index= 955 case= UJA <class 'str'>
<class 'list'> 8
--------------------------------------------------------
index= 956 case= VJA <class 'str'>
<class 'list'> 3
--------------------------------------------------------
index= 957 case= WJA <class 'str'>
<class 'list'> 17
--------------------------------------------------------
index= 958 case= XJA <class 'str'>
<class 'list'> 20
--------------------------------------------------------
index= 959 case= YJA <class 'str'>
<class 'list'> 12
--------------------------------------------------------
index= 960 case= ZJA <class 'str'>
<class 'list'> 23
--------------------------------------------------------
index= 961 case= AKA <class 'str'>
<class 'list'> 3
--------------------------------------------------------
index= 962 case= BKA <class 'str'>
<class 'list'> 19
--------------------------------------------------------
index= 963 case= CKA <class 'str'>
<class 'list'> 17
-------------

index= 1030 case= SMA <class 'str'>
<class 'list'> 16
--------------------------------------------------------
index= 1031 case= TMA <class 'str'>
<class 'list'> 10
--------------------------------------------------------
index= 1032 case= UMA <class 'str'>
<class 'list'> 14
--------------------------------------------------------
index= 1033 case= VMA <class 'str'>
<class 'list'> 3
--------------------------------------------------------
index= 1034 case= WMA <class 'str'>
<class 'list'> 5
--------------------------------------------------------
index= 1035 case= XMA <class 'str'>
<class 'list'> 3
--------------------------------------------------------
index= 1036 case= YMA <class 'str'>
<class 'list'> 9
--------------------------------------------------------
index= 1037 case= ZMA <class 'str'>
<class 'list'> 66
--------------------------------------------------------
index= 1038 case= ANA <class 'str'>
<class 'list'> 9
--------------------------------------------------------
index=

# validation

In [15]:
my_processed_df = pd.read_csv('./processed_csv/bpic13-0.3-2.csv')
processed_df = pd.read_csv ('./processed_bpic13-0.3-2.csv')

assert_frame_equal(my_processed_df, processed_df)

# SepsisCases-0.1-1.csv

In [45]:
SepsisCases_processed_df = pd.read_csv('./original_csv/formal/SepsisCases-0.1-1.csv')

In [46]:
SepsisCases_processed_df

,order,concept:name,label,caseid,name,timestamp
0,1,A,"{'anomaly': 'Late', 'attr': {'shift_from': 9, ...",A,ER Registration,2014-10-22T11:15:41.000+02:00
1,2,A,"{'anomaly': 'Late', 'attr': {'shift_from': 9, ...",A,Leucocytes,2014-10-22T11:27:00.000+02:00
2,3,A,"{'anomaly': 'Late', 'attr': {'shift_from': 9, ...",A,CRP,2014-10-22T11:27:00.000+02:00
3,4,A,"{'anomaly': 'Late', 'attr': {'shift_from': 9, ...",A,LacticAcid,2014-10-22T11:27:00.000+02:00
4,5,A,"{'anomaly': 'Late', 'attr': {'shift_from': 9, ...",A,ER Triage,2014-10-22T11:33:37.000+02:00
...,...,...,...,...,...,...
15254,14,KNA,normal,KNA,CRP,2014-12-16T07:00:00.000+01:00
15255,15,KNA,normal,KNA,Release A,2014-12-16T17:00:00.000+01:00
15256,1,LNA,normal,LNA,ER Registration,2014-12-03T10:50:28.000+01:00
15257,2,LNA,normal,LNA,ER Triage,2014-12-03T10:54:19.000+01:00


In [48]:
type(SepsisCases_processed_df['caseid'])

pandas.core.series.Series

443 nan <class 'float'>

In [50]:
for index, value in SepsisCases_processed_df['caseid'].items():
    print(index, value, type(value))

0 A <class 'str'>
1 A <class 'str'>
2 A <class 'str'>
3 A <class 'str'>
4 A <class 'str'>
5 A <class 'str'>
6 A <class 'str'>
7 A <class 'str'>
8 A <class 'str'>
9 A <class 'str'>
10 A <class 'str'>
11 A <class 'str'>
12 A <class 'str'>
13 A <class 'str'>
14 A <class 'str'>
15 A <class 'str'>
16 A <class 'str'>
17 A <class 'str'>
18 A <class 'str'>
19 A <class 'str'>
20 A <class 'str'>
21 A <class 'str'>
22 B <class 'str'>
23 B <class 'str'>
24 B <class 'str'>
25 B <class 'str'>
26 B <class 'str'>
27 B <class 'str'>
28 B <class 'str'>
29 B <class 'str'>
30 B <class 'str'>
31 B <class 'str'>
32 B <class 'str'>
33 B <class 'str'>
34 C <class 'str'>
35 C <class 'str'>
36 C <class 'str'>
37 C <class 'str'>
38 C <class 'str'>
39 C <class 'str'>
40 C <class 'str'>
41 C <class 'str'>
42 C <class 'str'>
43 C <class 'str'>
44 C <class 'str'>
45 C <class 'str'>
46 C <class 'str'>
47 C <class 'str'>
48 D <class 'str'>
49 D <class 'str'>
50 D <class 'str'>
51 D <class 'str'>
52 D <class 'str'>
53 

1284 ED <class 'str'>
1285 ED <class 'str'>
1286 ED <class 'str'>
1287 ED <class 'str'>
1288 ED <class 'str'>
1289 ED <class 'str'>
1290 ED <class 'str'>
1291 ED <class 'str'>
1292 ED <class 'str'>
1293 ED <class 'str'>
1294 ED <class 'str'>
1295 ED <class 'str'>
1296 ED <class 'str'>
1297 ED <class 'str'>
1298 ED <class 'str'>
1299 ED <class 'str'>
1300 ED <class 'str'>
1301 ED <class 'str'>
1302 ED <class 'str'>
1303 ED <class 'str'>
1304 ED <class 'str'>
1305 ED <class 'str'>
1306 ED <class 'str'>
1307 ED <class 'str'>
1308 ED <class 'str'>
1309 ED <class 'str'>
1310 ED <class 'str'>
1311 ED <class 'str'>
1312 ED <class 'str'>
1313 FD <class 'str'>
1314 FD <class 'str'>
1315 FD <class 'str'>
1316 FD <class 'str'>
1317 FD <class 'str'>
1318 FD <class 'str'>
1319 FD <class 'str'>
1320 FD <class 'str'>
1321 FD <class 'str'>
1322 FD <class 'str'>
1323 FD <class 'str'>
1324 GD <class 'str'>
1325 GD <class 'str'>
1326 GD <class 'str'>
1327 GD <class 'str'>
1328 GD <class 'str'>
1329 GD <c

2614 LG <class 'str'>
2615 LG <class 'str'>
2616 LG <class 'str'>
2617 LG <class 'str'>
2618 LG <class 'str'>
2619 LG <class 'str'>
2620 LG <class 'str'>
2621 LG <class 'str'>
2622 LG <class 'str'>
2623 LG <class 'str'>
2624 LG <class 'str'>
2625 LG <class 'str'>
2626 LG <class 'str'>
2627 LG <class 'str'>
2628 LG <class 'str'>
2629 LG <class 'str'>
2630 MG <class 'str'>
2631 MG <class 'str'>
2632 MG <class 'str'>
2633 MG <class 'str'>
2634 MG <class 'str'>
2635 MG <class 'str'>
2636 NG <class 'str'>
2637 NG <class 'str'>
2638 NG <class 'str'>
2639 NG <class 'str'>
2640 NG <class 'str'>
2641 NG <class 'str'>
2642 OG <class 'str'>
2643 OG <class 'str'>
2644 OG <class 'str'>
2645 OG <class 'str'>
2646 OG <class 'str'>
2647 OG <class 'str'>
2648 OG <class 'str'>
2649 OG <class 'str'>
2650 OG <class 'str'>
2651 PG <class 'str'>
2652 PG <class 'str'>
2653 PG <class 'str'>
2654 PG <class 'str'>
2655 PG <class 'str'>
2656 PG <class 'str'>
2657 PG <class 'str'>
2658 PG <class 'str'>
2659 PG <c

3789 RJ <class 'str'>
3790 SJ <class 'str'>
3791 SJ <class 'str'>
3792 SJ <class 'str'>
3793 SJ <class 'str'>
3794 SJ <class 'str'>
3795 SJ <class 'str'>
3796 SJ <class 'str'>
3797 SJ <class 'str'>
3798 SJ <class 'str'>
3799 SJ <class 'str'>
3800 SJ <class 'str'>
3801 SJ <class 'str'>
3802 SJ <class 'str'>
3803 SJ <class 'str'>
3804 SJ <class 'str'>
3805 SJ <class 'str'>
3806 TJ <class 'str'>
3807 TJ <class 'str'>
3808 TJ <class 'str'>
3809 TJ <class 'str'>
3810 TJ <class 'str'>
3811 TJ <class 'str'>
3812 TJ <class 'str'>
3813 TJ <class 'str'>
3814 TJ <class 'str'>
3815 TJ <class 'str'>
3816 TJ <class 'str'>
3817 TJ <class 'str'>
3818 TJ <class 'str'>
3819 TJ <class 'str'>
3820 TJ <class 'str'>
3821 UJ <class 'str'>
3822 UJ <class 'str'>
3823 UJ <class 'str'>
3824 UJ <class 'str'>
3825 UJ <class 'str'>
3826 UJ <class 'str'>
3827 UJ <class 'str'>
3828 UJ <class 'str'>
3829 UJ <class 'str'>
3830 UJ <class 'str'>
3831 UJ <class 'str'>
3832 UJ <class 'str'>
3833 UJ <class 'str'>
3834 UJ <c

5283 AN <class 'str'>
5284 AN <class 'str'>
5285 AN <class 'str'>
5286 AN <class 'str'>
5287 AN <class 'str'>
5288 AN <class 'str'>
5289 BN <class 'str'>
5290 BN <class 'str'>
5291 BN <class 'str'>
5292 BN <class 'str'>
5293 BN <class 'str'>
5294 BN <class 'str'>
5295 BN <class 'str'>
5296 BN <class 'str'>
5297 BN <class 'str'>
5298 BN <class 'str'>
5299 BN <class 'str'>
5300 BN <class 'str'>
5301 BN <class 'str'>
5302 BN <class 'str'>
5303 BN <class 'str'>
5304 BN <class 'str'>
5305 BN <class 'str'>
5306 BN <class 'str'>
5307 BN <class 'str'>
5308 BN <class 'str'>
5309 BN <class 'str'>
5310 BN <class 'str'>
5311 BN <class 'str'>
5312 BN <class 'str'>
5313 BN <class 'str'>
5314 BN <class 'str'>
5315 CN <class 'str'>
5316 CN <class 'str'>
5317 CN <class 'str'>
5318 CN <class 'str'>
5319 CN <class 'str'>
5320 CN <class 'str'>
5321 CN <class 'str'>
5322 CN <class 'str'>
5323 CN <class 'str'>
5324 CN <class 'str'>
5325 CN <class 'str'>
5326 DN <class 'str'>
5327 DN <class 'str'>
5328 DN <c

...

9616 IY <class 'str'>
9617 IY <class 'str'>
9618 IY <class 'str'>
9619 IY <class 'str'>
9620 IY <class 'str'>
9621 IY <class 'str'>
9622 IY <class 'str'>
9623 IY <class 'str'>
9624 IY <class 'str'>
9625 IY <class 'str'>
9626 IY <class 'str'>
9627 IY <class 'str'>
9628 IY <class 'str'>
9629 IY <class 'str'>
9630 IY <class 'str'>
9631 IY <class 'str'>
9632 JY <class 'str'>
9633 JY <class 'str'>
9634 JY <class 'str'>
9635 JY <class 'str'>
9636 JY <class 'str'>
9637 JY <class 'str'>
9638 JY <class 'str'>
9639 JY <class 'str'>
9640 JY <class 'str'>
9641 JY <class 'str'>
9642 JY <class 'str'>
9643 JY <class 'str'>
9644 JY <class 'str'>
9645 JY <class 'str'>
9646 JY <class 'str'>
9647 JY <class 'str'>
9648 JY <class 'str'>
9649 JY <class 'str'>
9650 KY <class 'str'>
9651 KY <class 'str'>
9652 KY <class 'str'>
9653 KY <class 'str'>
9654 KY <class 'str'>
9655 LY <class 'str'>
9656 LY <class 'str'>
9657 LY <class 'str'>
9658 MY <class 'str'>
9659 MY <class 'str'>
9660 MY <class 'str'>
9661 MY <c

10949 XBA <class 'str'>
10950 XBA <class 'str'>
10951 XBA <class 'str'>
10952 XBA <class 'str'>
10953 XBA <class 'str'>
10954 XBA <class 'str'>
10955 XBA <class 'str'>
10956 XBA <class 'str'>
10957 XBA <class 'str'>
10958 XBA <class 'str'>
10959 XBA <class 'str'>
10960 XBA <class 'str'>
10961 XBA <class 'str'>
10962 XBA <class 'str'>
10963 XBA <class 'str'>
10964 XBA <class 'str'>
10965 YBA <class 'str'>
10966 YBA <class 'str'>
10967 YBA <class 'str'>
10968 YBA <class 'str'>
10969 YBA <class 'str'>
10970 YBA <class 'str'>
10971 YBA <class 'str'>
10972 YBA <class 'str'>
10973 YBA <class 'str'>
10974 YBA <class 'str'>
10975 YBA <class 'str'>
10976 YBA <class 'str'>
10977 YBA <class 'str'>
10978 YBA <class 'str'>
10979 ZBA <class 'str'>
10980 ZBA <class 'str'>
10981 ZBA <class 'str'>
10982 ZBA <class 'str'>
10983 ZBA <class 'str'>
10984 ZBA <class 'str'>
10985 ZBA <class 'str'>
10986 ZBA <class 'str'>
10987 ZBA <class 'str'>
10988 ZBA <class 'str'>
10989 ZBA <class 'str'>
10990 ACA <class

12407 DGA <class 'str'>
12408 DGA <class 'str'>
12409 DGA <class 'str'>
12410 DGA <class 'str'>
12411 DGA <class 'str'>
12412 DGA <class 'str'>
12413 DGA <class 'str'>
12414 DGA <class 'str'>
12415 DGA <class 'str'>
12416 DGA <class 'str'>
12417 DGA <class 'str'>
12418 DGA <class 'str'>
12419 DGA <class 'str'>
12420 DGA <class 'str'>
12421 DGA <class 'str'>
12422 DGA <class 'str'>
12423 EGA <class 'str'>
12424 EGA <class 'str'>
12425 EGA <class 'str'>
12426 EGA <class 'str'>
12427 EGA <class 'str'>
12428 EGA <class 'str'>
12429 EGA <class 'str'>
12430 EGA <class 'str'>
12431 EGA <class 'str'>
12432 EGA <class 'str'>
12433 EGA <class 'str'>
12434 EGA <class 'str'>
12435 EGA <class 'str'>
12436 FGA <class 'str'>
12437 FGA <class 'str'>
12438 FGA <class 'str'>
12439 FGA <class 'str'>
12440 FGA <class 'str'>
12441 FGA <class 'str'>
12442 FGA <class 'str'>
12443 FGA <class 'str'>
12444 FGA <class 'str'>
12445 FGA <class 'str'>
12446 FGA <class 'str'>
12447 GGA <class 'str'>
12448 GGA <class

13782 IJA <class 'str'>
13783 IJA <class 'str'>
13784 IJA <class 'str'>
13785 IJA <class 'str'>
13786 IJA <class 'str'>
13787 IJA <class 'str'>
13788 IJA <class 'str'>
13789 IJA <class 'str'>
13790 IJA <class 'str'>
13791 JJA <class 'str'>
13792 JJA <class 'str'>
13793 JJA <class 'str'>
13794 JJA <class 'str'>
13795 JJA <class 'str'>
13796 JJA <class 'str'>
13797 JJA <class 'str'>
13798 JJA <class 'str'>
13799 JJA <class 'str'>
13800 KJA <class 'str'>
13801 KJA <class 'str'>
13802 KJA <class 'str'>
13803 KJA <class 'str'>
13804 KJA <class 'str'>
13805 LJA <class 'str'>
13806 LJA <class 'str'>
13807 LJA <class 'str'>
13808 LJA <class 'str'>
13809 LJA <class 'str'>
13810 LJA <class 'str'>
13811 LJA <class 'str'>
13812 LJA <class 'str'>
13813 MJA <class 'str'>
13814 MJA <class 'str'>
13815 MJA <class 'str'>
13816 MJA <class 'str'>
13817 MJA <class 'str'>
13818 MJA <class 'str'>
13819 MJA <class 'str'>
13820 MJA <class 'str'>
13821 NJA <class 'str'>
13822 NJA <class 'str'>
13823 NJA <class

15255 KNA <class 'str'>
15256 LNA <class 'str'>
15257 LNA <class 'str'>
15258 LNA <class 'str'>


In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
 
# data_frame = np.load('a.npy',allow_pickle=True)
# print(data_frame.columns)
 
df1 = pd.DataFrame({'one' : [1,2,3,pd.NA]})
df2 = pd.DataFrame({'one' : [1,2,3,np.nan]})
df3 = pd.DataFrame({'one' : [1,2,3,None]})

In [63]:
np.nan, type(np.nan)

(nan, float)

In [74]:
np.nan == np.nan

False

In [75]:
str(np.nan)

'nan'